In [1]:
import requests
import pandas as pd

In [ ]:
# https://docs.apidq.io/swagger/#/address/addressClean

#### read excel

In [2]:
rename_dict = {
    'Город': 'city',
    'Населенный пункт': 'city',
    #
    'Адреса': 'address',
    'Адрес': 'address',
    #
    'Название': 'name'
}

In [3]:
#file = 'ЛПУ по городам 20.09.2022 (2) (1) (1).xlsx'
file = 'Лаборатории_по_городам_от_03_09_22_1.xlsx'

df = pd.read_excel(file)
df.rename(columns=rename_dict, inplace=True)

In [4]:
df.head()

,city,address,Название лаборатории
0,г. Москва,"город Москва, Шоссе Энтузиастов, дом 22/18",Инвитро
1,г. Москва,"город Москва, улица Солдатская, дом 10, корп. 2",Инвитро
2,г. Москва,"город Москва, 1-ый Кожуховский пр-д, дом 9",Инвитро
3,г. Москва,"город Москва, улица Профсоюзная, дом 5/9",Инвитро
4,г. Москва,"город Москва, улица Староалексеевская, дом 8",Инвитро


#### API

In [5]:
url = 'https://api.apidq.io/v1/clean/address'

In [20]:
data = {
  "query": "г. Москва	город Москва, Шоссе Энтузиастов, дом 22/18",
  "countryCode": "RU"
}

In [21]:
r = requests.post(url, json=data)

In [22]:
true = True
false = False


In [23]:
d = eval(r.text)

In [24]:
d['region']['fullName']

'г Москва'

In [25]:
d['addressFull']

'г Москва, Энтузиастов ш, дом 22/18'

In [26]:
d['city']['fullName']

''

In [27]:
d['settlement']['fullName']

''

In [28]:
d

{'original': 'г. москва\tгород москва, шоссе энтузиастов, дом 22/18',
 'addressFull': 'г Москва, Энтузиастов ш, дом 22/18',
 'address': 'г Москва, Энтузиастов ш',
 'postcodeIn': '',
 'postcode': '111123',
 'region': {'fullName': 'г Москва',
  'name': 'Москва',
  'type': 'г',
  'codes': {'fias': '0c5b2444-70a0-4932-980c-b4dc0d3f02b5',
   'ga': 'RU0770000000000000000000000',
   'osm': 'R102269',
   'gar': '1405113',
   'kladr': '7700000000000'}},
 'area': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'city': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'cityArea': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'settlement': {'fullName': '',
  'name': '',
  'type': '',
  'codes': {'fias': '', 'ga': '', 'osm': '', 'gar': '', 'kladr': ''}},
 'planStructure': {'fullName': '',
  'nam

#### clean address

In [29]:
true = True
false = False

df['address_clean'] = ''
df['city_clean'] = ''

big_cities = [
    'Москва',
    'Санкт-Петербург',
    'Севастополь'
]

In [30]:
for i in range(df.shape[0]):
    temp_addr = ''
    addressFull = ''
    bigcity = ''
    city = ''
    settlement = ''
    
    if df['city'][i] not in df['address']:
        temp_addr = df['city'][i] + ', ' + df['address'][i]
    else:
        temp_addr = df['address'][i]
        
    data = {
      "query": temp_addr,
      "countryCode": "RU"
    }
    
    r = requests.post(url, json=data)
    
    d = eval(r.text)
    
    try:
        #bigcity = d['region']['fullName']
        bigcity = d['region']['name']
    except:
        bigcity = ''
        
    try:
        addressFull = d['addressFull']
    except:
        addressFull = 'Не найдено'
    
    try:
        #city = d['city']['fullName']
        city = d['city']['name']
    except:
        addressFull = ''
    
    try:
        #settlement = d['settlement']['fullName']
        settlement = d['settlement']['name']
    except:
        settlement = 'Не найдено'
    
    df['address_clean'][i] = addressFull
    
    if bigcity in big_cities:
        df['city_clean'][i] = bigcity
    elif city:
        df['city_clean'][i] = city
    else:
        df['city_clean'][i] = settlement

In [31]:
#df.to_excel('test1.xlsx')

In [32]:
df

,city,address,Название лаборатории,address_clean,city_clean
0,г. Москва,"город Москва, Шоссе Энтузиастов, дом 22/18",Инвитро,"г Москва, Энтузиастов ш, дом 22/18",Москва
1,г. Москва,"город Москва, улица Солдатская, дом 10, корп. 2",Инвитро,"г Москва, ул Солдатская, дом 10, корпус 2",Москва
2,г. Москва,"город Москва, 1-ый Кожуховский пр-д, дом 9",Инвитро,"г Москва, 1-й Кожуховский проезд, дом 9",Москва
3,г. Москва,"город Москва, улица Профсоюзная, дом 5/9",Инвитро,"г Москва, ул Профсоюзная, дом 5/9",Москва
4,г. Москва,"город Москва, улица Староалексеевская, дом 8",Инвитро,"г Москва, ул Староалексеевская, дом 8",Москва
...,...,...,...,...,...
3086,Уссурийск,"Чичерина, 83",Юнилаб,"Приморский край, г Уссурийск, ул Чичерина, дом 83",Уссурийск
3087,Уссурийск,"Пионерская, 19",Юнилаб,"Приморский край, г Уссурийск, ул Пионерская, д...",Уссурийск
3088,Фокино,"Клубная,15а",Юнилаб,"Приморский край, г Фокино, ул Клубная, дом 15а",Фокино
3089,Ю Сахалинск,"Чехова, 87",Юнилаб,"Сахалинская обл, г Южно-Сахалинск, ул Чехова, ...",Южно-Сахалинск


In [42]:
d

{'code': 5, 'message': 'Ничего не найдено', 'details': []}

In [33]:
df.to_excel('clean_name_' + file, index=False)

In [ ]:
#21_02